In [1]:
import pandas as pd
import numpy as np
import datetime
pd.set_option('display.max_columns', 50)

In [2]:
asset_name = 'SPX'
vol_name = 'VIX'
asset_price = pd.read_csv('./raw/price/' +asset_name +'.csv')
asset_vol = pd.read_csv('./raw/vol/' +vol_name +'.csv')

In [3]:
# Returns a dataframe with empty and unchanging columns removed
def drop_unchanging(df):
    #remove fully empty columns
    no_empty = df.dropna(axis=1, how='all')
    #Drop all columns that only have one non-empty value
    to_drop = [col for col in no_empty.columns if ((no_empty[col].dtype == np.float64 or
               no_empty[col].dtype == np.int64) and sum(~np.isnan(no_empty[col].unique())) < 2)]
    return no_empty.drop(to_drop, axis=1) #remove columns with unchanging values

## Joining Data Together

### Hourly Price Data Cleaning

Note the TRMI is recorded in the GMT timezone. The hourly TRMI is a time series containing every hour for each day. The price data here is also recorded in GMT (adding the offset to the time converts back to market hours). Because the price data can only be over market hours, there will be empty cells.

Here we simplify some of the Russell 2000 data fields:

In [4]:
#Remove asset id column, type column, and unchanging columns:
new_price = drop_unchanging(asset_price.drop(['#RIC', 'Type'], 1))

mon_str_to_num = {'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
                  'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}

#Convert to all numeric date and reverse to match TRMI format
date_col = asset_price['Date[G]'].map(lambda w: (w[7:] + '-' + mon_str_to_num[w[3:6]] + '-' + w[:2]))
hour_col = asset_price['Time[G]'].map(lambda w: int(w[:2]))
date_hour_col = date_col.map(str) + 'T' + hour_col.map(lambda h: str(h).zfill(2)) + ':00'

new_price.insert(0, 'id', date_hour_col)
new_price.insert(1, 'date', date_col)
new_price.insert(2, 'hour', hour_col)
new_price.insert(3, 'gmt_offset', asset_price['GMT Offset'])
new_price = new_price.drop(['Date[G]', 'Time[G]', 'GMT Offset'], 1) #Drop redundant date/time columns
new_price.set_index('id', drop=True, inplace=True)

new_price.head()

,date,hour,gmt_offset,Open,High,Low,Last,Volume,Ave. Price,No. Trades,Open Bid,High Bid,Low Bid,Close Bid,No. Bids,Open Ask,High Ask,Low Ask,Close Ask,No. Asks
id,,,,,,,,,,,,,,,,,,,,
1998-01-02T10:00,1998-01-02,10,-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,1
1998-01-02T14:00,1998-01-02,14,-6,970.43,974.02,970.43,970.82,0.0,972.2026,121.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0
1998-01-02T15:00,1998-01-02,15,-6,970.72,970.72,965.97,969.55,0.0,968.2946,237.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0
1998-01-02T16:00,1998-01-02,16,-6,969.63,970.82,968.92,968.97,0.0,969.6750,240.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0
1998-01-02T17:00,1998-01-02,17,-6,969.00,971.26,969.00,971.12,0.0,970.5000,240.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0


### Hourly Volatility Data Cleaning

In [5]:
#Remove asset id column, type column, and unchanging columns:
new_vol = drop_unchanging(asset_vol.drop(['#RIC', 'Type'], 1))

mon_str_to_num = {'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
                  'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}

#Convert to all numeric date and reverse to match TRMI format
date_col = new_vol['Date[G]'].map(lambda w: (w[7:] + '-' + mon_str_to_num[w[3:6]] + '-' + w[:2]))
hour_col = new_vol['Time[G]'].map(lambda w: int(w[:2]))
date_hour_col = date_col.map(str) + 'T' + hour_col.map(lambda h: str(h).zfill(2)) + ':00'

new_vol.insert(0, 'id', date_hour_col)
new_vol.insert(1, 'date', date_col)
new_vol.insert(2, 'hour', hour_col)

#Don't add gmt_offset, because we want to use the gmt offset associated with the pricing data
#(ie we want to use eastern time, not central time)
#new_r2k_volatility.insert(3, 'gmt_offset', r2k_price['GMT Offset'])
new_vol = new_vol.drop(['Date[G]', 'Time[G]', 'GMT Offset'], 1) #Drop redundant date/time columns
new_vol.set_index('id', drop=True, inplace=True)

new_vol.head()

,date,hour,Open,High,Low,Last,Ave. Price,No. Trades,No. Bids,No. Asks
id,,,,,,,,,,
1998-01-02T10:00,1998-01-02,10,NaN,NaN,NaN,NaN,NaN,NaN,1,1
1998-01-02T15:00,1998-01-02,15,25.80,26.47,25.28,25.48,25.8066,59.0,0,0
1998-01-02T16:00,1998-01-02,16,25.33,25.66,25.10,25.63,25.3995,60.0,0,0
1998-01-02T17:00,1998-01-02,17,25.63,25.66,25.04,25.15,25.3143,60.0,0,0
1998-01-02T18:00,1998-01-02,18,25.18,25.49,24.85,25.30,25.1930,60.0,0,0


###  Join Volatility Data

In [6]:
price_vol = new_price.merge(new_vol, how='left', on=['date', 'hour'],
                                                 left_index=True, right_index=True, suffixes=('', '_' +vol_name))

In [7]:
price_vol.head()

,date,hour,gmt_offset,Open,High,Low,Last,Volume,Ave. Price,No. Trades,Open Bid,High Bid,Low Bid,Close Bid,No. Bids,Open Ask,High Ask,Low Ask,Close Ask,No. Asks,Open_VIX,High_VIX,Low_VIX,Last_VIX,Ave. Price_VIX,No. Trades_VIX,No. Bids_VIX,No. Asks_VIX
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1998-01-02T10:00,1998-01-02,10,-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
1998-01-02T14:00,1998-01-02,14,-6,970.43,974.02,970.43,970.82,0.0,972.2026,121.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-02T15:00,1998-01-02,15,-6,970.72,970.72,965.97,969.55,0.0,968.2946,237.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,25.80,26.47,25.28,25.48,25.8066,59.0,0.0,0.0
1998-01-02T16:00,1998-01-02,16,-6,969.63,970.82,968.92,968.97,0.0,969.6750,240.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,25.33,25.66,25.10,25.63,25.3995,60.0,0.0,0.0
1998-01-02T17:00,1998-01-02,17,-6,969.00,971.26,969.00,971.12,0.0,970.5000,240.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,25.63,25.66,25.04,25.15,25.3143,60.0,0.0,0.0


## Add Day of Week, Day, Month, Year

In [8]:
year_col = price_vol['date'].map(lambda w: int(w[:4]))
month_col = price_vol['date'].map(lambda w: int(w[5:7]))
day_col = price_vol['date'].map(lambda w: int(w[8:10]))
dow_col = price_vol['date'].map(lambda w: datetime.datetime.strptime(w, '%Y-%m-%d').weekday())

price_vol.insert(3, 'dow', dow_col)
price_vol.insert(4, 'day', day_col)
price_vol.insert(5, 'month', month_col)
price_vol.insert(6, 'year', year_col)

price_vol.head()

,date,hour,gmt_offset,dow,day,month,year,Open,High,Low,Last,Volume,Ave. Price,No. Trades,Open Bid,High Bid,Low Bid,Close Bid,No. Bids,Open Ask,High Ask,Low Ask,Close Ask,No. Asks,Open_VIX,High_VIX,Low_VIX,Last_VIX,Ave. Price_VIX,No. Trades_VIX,No. Bids_VIX,No. Asks_VIX
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1998-01-02T10:00,1998-01-02,10,-6,4,2,1,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
1998-01-02T14:00,1998-01-02,14,-6,4,2,1,1998,970.43,974.02,970.43,970.82,0.0,972.2026,121.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-02T15:00,1998-01-02,15,-6,4,2,1,1998,970.72,970.72,965.97,969.55,0.0,968.2946,237.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,25.80,26.47,25.28,25.48,25.8066,59.0,0.0,0.0
1998-01-02T16:00,1998-01-02,16,-6,4,2,1,1998,969.63,970.82,968.92,968.97,0.0,969.6750,240.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,25.33,25.66,25.10,25.63,25.3995,60.0,0.0,0.0
1998-01-02T17:00,1998-01-02,17,-6,4,2,1,1998,969.00,971.26,969.00,971.12,0.0,970.5000,240.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,25.63,25.66,25.04,25.15,25.3143,60.0,0.0,0.0


## Add Bid Ask Spreads

In [9]:
price_vol.insert(23, 'Open_BA_Spread', price_vol['Open Ask'] - price_vol['Open Bid'])
price_vol.insert(24, 'Max_BA_Spread', price_vol['High Ask'] - price_vol['Low Bid'])
price_vol.insert(25, 'Min_BA_Spread', price_vol['Low Ask'] - price_vol['High Bid'])
price_vol.insert(26, 'Close_BA_Spread', price_vol['Close Ask'] - price_vol['Close Bid'])

## Add Targets/Labels

### Regression Targets

In [10]:
price_vol['gross_return'] = price_vol['Last'] / price_vol['Open']
price_vol['net_return'] = price_vol['gross_return'] - 1
price_vol['log_return'] = np.log(price_vol['gross_return'].astype('float64'))

### Classification Labels

In [11]:
price_vol['price_dir'] = np.sign(price_vol['net_return'])
price_vol['long_bias_price_dir'] = price_vol['price_dir'].replace(to_replace=0.0, value=1.0) # replace sw label with up label

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: invalid value encountered in sign
  if __name__ == '__main__':


In [12]:
#price_vol_chopped[price_vol_chopped['price_dir'].shift(1)==0]['price_dir'].value_counts()

## Dropping Rows and Columns

### Drop Premarket and Postmarket Hours

Most of the data is over nontrading hours (or pre/post market hours). This data needs to be dropped. Additionally, dropping the last hour (which is post market) may improve performance. Here we see most of the time premarket and postmarket trading shows no change or trading at all:

In [13]:
print('Before 8AM:')
price_vol[(price_vol['hour'] + price_vol['gmt_offset']) < 8]['price_dir'].value_counts()

Before 8AM:


1.0    2
0.0    1
Name: price_dir, dtype: int64

In [14]:
print('After 3PM:')
price_vol[(price_vol['hour'] + price_vol['gmt_offset']) > 15]['price_dir'].value_counts()

After 3PM:


 0.0    874
-1.0     14
 1.0     10
Name: price_dir, dtype: int64

In [15]:
price_vol_chopped = price_vol.copy()
price_vol_chopped = price_vol_chopped[(price_vol_chopped['hour'] + price_vol_chopped['gmt_offset']) > 7]  # Drop premarket
price_vol_chopped = price_vol_chopped[(price_vol_chopped['hour'] + price_vol_chopped['gmt_offset']) < 16] # Drop postmarket

### Drop all Other Nontrading Periods

Drop weekends, holidays, early close trading days, etc:

In [16]:
price_vol_chopped.dropna(axis=0, how='any', subset=['Open', 'High', 'Low', 'Last'], inplace=True)

### Drop Unneeded Columns

In [17]:
price_vol_chopped.drop('Volume', axis=1, inplace=True)
price_vol_chopped.drop('No. Bids', axis=1, inplace=True)
price_vol_chopped.drop('No. Asks', axis=1, inplace=True)
price_vol_chopped.drop('No. Bids_' + vol_name, axis=1, inplace=True)
price_vol_chopped.drop('No. Asks_' + vol_name, axis=1, inplace=True)

In [18]:
price_vol_chopped.columns

Index(['date', 'hour', 'gmt_offset', 'dow', 'day', 'month', 'year', 'Open',
       'High', 'Low', 'Last', 'Ave. Price', 'No. Trades', 'Open Bid',
       'High Bid', 'Low Bid', 'Close Bid', 'Open Ask', 'High Ask', 'Low Ask',
       'Close Ask', 'Open_BA_Spread', 'Max_BA_Spread', 'Min_BA_Spread',
       'Close_BA_Spread', 'Open_VIX', 'High_VIX', 'Low_VIX', 'Last_VIX',
       'Ave. Price_VIX', 'No. Trades_VIX', 'gross_return', 'net_return',
       'log_return', 'price_dir', 'long_bias_price_dir'],
      dtype='object')

## Front Fill Volatility Data

Replace zeros with nans first:

In [19]:
price_vol_chopped['Open_VIX'].replace(to_replace=0.0, value=np.nan, inplace=True)
price_vol_chopped['High_VIX'].replace(to_replace=0.0, value=np.nan, inplace=True)
price_vol_chopped['Low_VIX'].replace(to_replace=0.0, value=np.nan, inplace=True)
price_vol_chopped['Last_VIX'].replace(to_replace=0.0, value=np.nan, inplace=True)
price_vol_chopped['Ave. Price_VIX'].replace(to_replace=0.0, value=np.nan, inplace=True)
price_vol_chopped['No. Trades_VIX'].replace(to_replace=0.0, value=np.nan, inplace=True)

Replace all nans with padded / frontfilled values

In [20]:
price_vol_chopped['Open_VIX'].fillna(method='ffill', inplace=True)
price_vol_chopped['High_VIX'].fillna(method='ffill', inplace=True)
price_vol_chopped['Low_VIX'].fillna(method='ffill', inplace=True)
price_vol_chopped['Last_VIX'].fillna(method='ffill', inplace=True)
price_vol_chopped['Ave. Price_VIX'].fillna(method='ffill', inplace=True)
price_vol_chopped['No. Trades_VIX'].fillna(method='ffill', inplace=True)

In [21]:
assert(price_vol_chopped[price_vol_chopped['Open_VIX'] == 0.0]['Open_VIX'].empty)

## Save Cleaned Data to File

In [22]:
price_vol_chopped.to_csv('./proc/' +asset_name +'-proc' +'.csv')

In [23]:
len(price_vol_chopped[price_vol_chopped['price_dir']==0.0])

124

## Feature Extaction

At this point we have the raw materials to transform our data into features more amenable to machine learning. We will transform the chosen time series into multiple moving averages. We'll also remove the gmt_offset column and add the is_dst feature column.

In [24]:
feature_matrix = price_vol_chopped[['date', 'hour', 'gmt_offset', 'dow', 'day', 'month',  'Open', 'Open_BA_Spread', 'Open_VIX']]
dst_offset = max(list(feature_matrix['gmt_offset'].unique()))
is_dst = feature_matrix['gmt_offset'].map(lambda offset: (offset == dst_offset) * 1)
feature_matrix.insert(2, 'is_dst', is_dst)
feature_matrix = feature_matrix.drop('gmt_offset', axis=1)

In [25]:
# Open based series do not need to be shifted forward (we have access to them at decision time)
ma_features = ['Open', 'Open_BA_Spread', 'Open_VIX']

# These need to be shifted because we don't have them at decision time
ma_shift_features = ['High', 'Low', 'Last', 'Ave. Price', 'No. Trades', 'Max_BA_Spread', 'Min_BA_Spread', 'Close_BA_Spread',
                     'High_VIX', 'Low_VIX', 'Last_VIX', 'Ave. Price_VIX', 'No. Trades_VIX', 'net_return', 'log_return',
                     'price_dir']

for ma_period in [2, 4, 8, 16, 40, 80, 160, 240]:
    sfx = '_' +str(ma_period) +'h_MA'
    ma = price_vol_chopped[ma_features].rolling(window=ma_period, min_periods=1, win_type='boxcar').mean().add_suffix(sfx)
    ma_shift = price_vol_chopped[ma_shift_features].rolling(window=ma_period, min_periods=1, win_type='boxcar').mean().shift(1).add_suffix(sfx)

    feature_matrix = feature_matrix.merge(ma, how='left', left_index=True, right_index=True)
    feature_matrix = feature_matrix.merge(ma_shift, how='left', left_index=True, right_index=True)

### Add Label Data

In [26]:
feature_matrix = feature_matrix.merge(price_vol_chopped[['net_return', 'log_return', 'price_dir', 'long_bias_price_dir']],
                                      how='left', left_index=True, right_index=True)

### Drop Data Before 8/2/1999

Bid/Ask data is not available before 7/2/1999 (at least for the S&P 500). We have to drop all rows a month from when we have this data because our longest moving average period is 240 hours (one month):

In [27]:
feature_matrix = feature_matrix[feature_matrix['date'] > '1999-08-01']

## Save Final Feature Matrix to File

In [28]:
feature_matrix.to_csv('./feat/' +asset_name +'-feat' +'.csv')